In [1]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'notebooks'...
remote: Enumerating objects: 515, done.
remote: Counting objects: 100% (515/515), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 515 (delta 245), reused 478 (delta 231), pack-reused 0
Receiving objects: 100% (515/515), 29.39 MiB | 27.71 MiB/s, done.
Resolving deltas: 100% (245/245), done.
/kaggle/working/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [2]:
#hide
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Settings > Accelerator and select GPU.
Using transformers v4.11.3
Using datasets v1.16.1


# Transformer Anatomy

Encoder-decoder architecture of the transformer, with the encoder shown in the upper half of the figure and the decoder in the lower half.

<img alt="transformer-encoder-decoder" caption="Encoder-decoder architecture of the transformer, with the encoder shown in the upper half of the figure and the decoder in the lower half" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_transformer-encoder-decoder.png" id="transformer-encoder-decoder"/>

## The Encoder

The transformer's encoder consists of many encoder layers stacked next to each other. As illustrated in the figure below, each encoder layer receives a sequence of enbeddings and feeds them through the following sublayers:
* A multi-head self-attention layer
* A fully connected feed-forward layer that is applied to each input embedding

> The main role of the encoder stack is to "update" the input embeddings to produce representations that encode some contextual information in the sequence.

<img alt="encoder-zoom" caption="Zooming into the encoder layer" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_encoder-zoom.png" id="encoder-zoom"/>

### Self-Attention

* The main idea behind self-attention is that instead of using a fixed embedding for each token, we can use the whole sequence to compute weighted average of each embedding. In other words, given a sequence of embeddings, self-attention produces a sequence of new embeddings where each new embedding is a linear combination of all original embedding vectors.
* The following diagram shows how self-attention updates raw token embeddings (upper) into contextualized embeddings (lower) to create contextualized representations that incorporate information from the whole sequence.

<img alt="Contextualized embeddings" caption="Diagram showing how self-attention updates raw token embeddings (upper) into contextualized embeddings (lower) to create representations that incorporate information from the whole sequence" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_contextualized-embedding.png" id="contextualized-embeddings"/>

Let's now take a look at how we can calculate the attention weights.

### Scaled dot-product attention

* There are several ways to implement a self-attention layer, but the most common one is scaled dot-product attiontion. It includes four main steps:
1. Project each token embedding into three vectors: key, query, value
2. Compute attention scores
3. Compute attention weights by applying softmax
4. For a specific token, multiply attention weight by value vector to get the updated form of token embedding.

* The following diagram represents the operations to compute scaled dot-product attention. Let's implement it:

<img alt="Operations in scaled dot-product attention" height="125" caption="Operations in scaled dot-product attention" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_attention-ops.png" id="attention-ops"/>

#### 1. Tokensize the text and get input IDs:

In [4]:
# import the tokenizer
from transformers import AutoTokenizer
model_ckpt = "bert-base-uncased"
text = "time flies like an arrow"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [5]:
# use our tokenizer to get the input IDs
# for simplicity, exclued [CLS] and [SEP] tokens
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

#### 2. Generate Raw Embeddings (non-contextual embedding):
* First, let's instantiate an Embedding Layer. We can do this by using `torch.nn.Embedding` layer that acts as a lookup table for each input ID:

In [6]:
from torch import nn
from transformers import AutoConfig
# load config.json associated with bert-base-uncased checkpoint
config = AutoConfig.from_pretrained(model_ckpt)

# instantiate the embedding layer
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

* Now, we can generate our raw embedding vectors:
> NOTE: These vectors are raw, i.e., non-contextual and the position is NOT considered (no positional encoding)

````python
text = "time flies like an arrow"
````

In [8]:
# feed input IDs into the embedding layer to generate non-contextual embedding
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()
# [batch_size, seq_len, hidden_dim]

torch.Size([1, 5, 768])

In [9]:
inputs_embeds

tensor([[[-0.2034,  0.4988, -0.1213,  ...,  0.7915,  0.6356,  1.4725],
         [-1.9127,  0.8434,  1.6794,  ..., -0.5380,  1.0705,  2.0819],
         [-1.1683,  1.1313, -1.6481,  ...,  0.0725,  0.4111, -1.0515],
         [-0.0669, -0.9048, -1.3120,  ...,  0.9904, -1.1233,  0.3818],
         [ 1.0926, -0.4669, -1.0284,  ..., -0.3842, -0.2131, -0.0954]]],
       grad_fn=<EmbeddingBackward0>)

#### 3.Create query, key, and value vectors:
* Now, let's create the query, key, and value vectors and calculate the attention scores using dot product as the similarity function:
> **NOTE**: For simplicity, projection step for **K**,**Q**, and **V** is negelected for now. That is, **K**, **Q**, and **V** are the same as the **raw embedding vector**. Later we'll generate three variants of this **raw vector** by projecting it using different matrices (The weights of those matrices will changed during training)

In [23]:
import torch
from math import sqrt 

# K, Q, V are the same
query = key = value = inputs_embeds # shape [1, 5, 768]


#### 4.Calculate attention scores:

In [24]:
# calculate attention scores 
dim_k = key.size(-1) # 768
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k) ## takes two batches of matrices and multiplies 
                                                            # each matrix from the first batch with the corresponding matrix in the second batch
scores.size()

torch.Size([1, 5, 5])

In [22]:
query, key.transpose(1,2)

(tensor([[[-0.2034,  0.4988, -0.1213,  ...,  0.7915,  0.6356,  1.4725],
          [-1.9127,  0.8434,  1.6794,  ..., -0.5380,  1.0705,  2.0819],
          [-1.1683,  1.1313, -1.6481,  ...,  0.0725,  0.4111, -1.0515],
          [-0.0669, -0.9048, -1.3120,  ...,  0.9904, -1.1233,  0.3818],
          [ 1.0926, -0.4669, -1.0284,  ..., -0.3842, -0.2131, -0.0954]]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[[-0.2034, -1.9127, -1.1683, -0.0669,  1.0926],
          [ 0.4988,  0.8434,  1.1313, -0.9048, -0.4669],
          [-0.1213,  1.6794, -1.6481, -1.3120, -1.0284],
          ...,
          [ 0.7915, -0.5380,  0.0725,  0.9904, -0.3842],
          [ 0.6356,  1.0705,  0.4111, -1.1233, -0.2131],
          [ 1.4725,  2.0819, -1.0515,  0.3818, -0.0954]]],
        grad_fn=<TransposeBackward0>))

In [19]:
scores

tensor([[[25.3802, -1.4039, -1.3772, -1.3363,  0.9843],
         [-1.4039, 29.1417, -1.4569,  0.1180, -2.0591],
         [-1.3772, -1.4569, 26.2450, -0.9152,  0.4735],
         [-1.3363,  0.1180, -0.9152, 30.9603, -0.3323],
         [ 0.9843, -2.0591,  0.4735, -0.3323, 26.7355]]],
       grad_fn=<DivBackward0>)

#### 5. Calculate attention weights by applying the softmax function on the attention scores:

In [27]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

#### 6. Update the raw token embedding: mulitply attention weights by the values:

In [28]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

#### Wrap the previous steps in a function:

In [ ]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

#### Multi-headed attention

<img alt="Multi-head attention" height="125" caption="Multi-head attention" src="images/chapter03_multihead-attention.png" id="multihead-attention"/>

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

In [ ]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)    
attn_output.size() 

torch.Size([1, 5, 768])

In [ ]:
#hide_output
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

<IPython.core.display.Javascript object>

### The Feed-Forward Layer

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

In [ ]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()

torch.Size([1, 5, 768])

### Adding Layer Normalization

<img alt="Transformer layer normalization" height="500" caption="Different arrangements of layer normalization in a transformer encoder layer" src="images/chapter03_layer-norm.png" id="layer-norm"/>

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # Apply layer normalization and then copy input into query, key, value
        hidden_state = self.layer_norm_1(x)
        # Apply attention with a skip connection
        x = x + self.attention(hidden_state)
        # Apply feed-forward layer with a skip connection
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [ ]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encoder_layer(inputs_embeds).size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

### Positional Embeddings

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size, 
                                             config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Create position IDs for input sequence
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config) 
                                     for _ in range(config.num_hidden_layers)])

    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])

### Adding a Classification Head

In [ ]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        
    def forward(self, x):
        x = self.encoder(x)[:, 0, :] # select hidden state of [CLS] token
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [ ]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

torch.Size([1, 3])

## The Decoder

<img alt="Transformer decoder zoom" caption="Zooming into the transformer decoder layer" src="images/chapter03_decoder-zoom.png" id="decoder-zoom"/> 

In [ ]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [ ]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[26.8082,    -inf,    -inf,    -inf,    -inf],
         [-0.6981, 26.9043,    -inf,    -inf,    -inf],
         [-2.3190,  1.2928, 27.8710,    -inf,    -inf],
         [-0.5897,  0.3497, -0.3807, 27.5488,    -inf],
         [ 0.5275,  2.0493, -0.4869,  1.6100, 29.0893]]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights.bmm(value)

### Sidebar: Demystifying Encoder-Decoder Attention

### End sidebar

## Meet the Transformers

### The Transformer Tree of Life

<img alt="Transformer family tree" caption="An overview of some of the most prominent transformer architectures" src="images/chapter03_transformers-compact.png" id="family-tree"/>

### The Encoder Branch

### The Decoder Branch

### The Encoder-Decoder Branch

## Conclusion